In [9]:
%load_ext autoreload

In [50]:
import numpy as np
import os
import mcubes
import pydrake
from pydrake.geometry import SceneGraph
from pydrake.systems.framework import DiagramBuilder
from pydrake.common import FindResourceOrThrow
from pydrake.multibody.plant import MultibodyPlant, AddMultibodyPlantSceneGraph
from pydrake.multibody.parsing import Parser, LoadModelDirectives, ProcessModelDirectives
from pydrake.multibody.tree import RevoluteJoint

from pydrake.all import ConnectMeshcatVisualizer, InverseKinematics, RigidTransform

from meshcat import Visualizer
from functools import partial

ModuleNotFoundError: No module named 'mcubes'

In [16]:
# Setup meshcat
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=[])


In [25]:
vis = Visualizer(zmq_url=zmq_url)
vis.delete()
display(vis.jupyter_cell())

builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
parser = Parser(plant)
robot_asset = FindResourceOrThrow("drake/sandbox/planar2dof.urdf")
model =parser.AddModelFromFile(robot_asset)

#directives_file = FindResourceOrThrow("drake/sandbox/planar_iiwa_simple_collision_welded_gripper.yaml") \
#    if simple_collision else FindResourceOrThrow("drake/sandbox/planar_iiwa_dense_collision_welded_gripper.yaml")
#directives = LoadModelDirectives(directives_file)
#models = ProcessModelDirectives(directives, plant, parser)


q0 = [0.0, 0.0, 0.0]
index = 0
for joint_index in plant.GetJointIndices(model):
    joint = plant.get_mutable_joint(joint_index)
    if isinstance(joint, RevoluteJoint):
        joint.set_default_angle(q0[index])
        index += 1
plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("base", model), RigidTransform([0.0,0,0.0]))
plant.Finalize()

visualizer = ConnectMeshcatVisualizer(builder, scene_graph, zmq_url=zmq_url, delete_prefix_on_load=False)

diagram = builder.Build()
visualizer.load()
context = diagram.CreateDefaultContext()
plant_context = plant.GetMyContextFromRoot(context)
diagram.Publish(context)

jointindx = plant.GetJointIndices(model)
joints = []
for j in jointindx:
    joint = plant.get_mutable_joint(j)
    if isinstance(joint, RevoluteJoint):
        joints.append(joint)
    
def set_joint_ang(val, idx):
    joints[idx].set_angle(plant_context, val)
    diagram.Publish(context)
    
ik = InverseKinematics(plant, plant_context)
collision_constraint = ik.AddMinimumDistanceConstraint(0.001, 0.01)

def eval_cons(q0, q1, q2, c, tol):
        return float(c.evaluator().CheckSatisfied([q0, q1, q2], tol))
    
col_func_handle = partial(eval_cons, c=collision_constraint, tol=0.05)    
q_low = [-1.5, -3, -1.5]
q_high = [1.5, 3, 1.5]

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7008/static/


Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6009...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7008/static/
Connected to meshcat-server.


In [49]:
#marching cubes
N = 100
vertices, triangles = mcubes.marching_cubes_func(tuple(q_low), tuple(q_high), N, N, N, col_func_handle, 0.5)

NameError: name 'mcubes' is not defined

In [48]:
q_low = [-1.5, -3, -1.5]
q_high = [1.5, 3, 1.5]
q = [0.0, 0, 0]

set_joint_ang(q[0], 0)
set_joint_ang(q[1], 1)
set_joint_ang(q[2], 2)
collision_constraint.evaluator().CheckSatisfied(q, 0.001)
#check collision
#partial(eval, c=collision_constraint, tol=0.05)

True

In [10]:
joint.set_angle(plant_context, 0.5)
diagram.Publish(context)

In [88]:
partial(eval, c=collision_constraint, tol=0.05)

functools.partial(<built-in function eval>, c=<pydrake.solvers.mathematicalprogram.Binding[Constraint] object at 0x7fd326079cb0>, tol=0.05)

In [ ]:
vertices, triangles = mcubes.marching_cubes_func(
            tuple(low), tuple(up), N, N, N, 
            partial(eval, c=collision_constraint, tol=0.0), 0.5)

In [109]:
def eval(q0, q1, q2, c, tol):
        return float(c.evaluator().CheckSatisfied([q0, q1, q2], tol))


In [26]:
collision_constraint.evaluator().CheckSatisfied([0.15,2,0.9], 0.001)

False

In [94]:
directives_file = FindResourceOrThrow("drake/sandbox/planar2dof.urdf")



TypeError: eval() takes no keyword arguments

In [32]:
simple_collision = True
# gripper_welded = True
vis = Visualizer(zmq_url=zmq_url)
vis.delete()
display(vis.jupyter_cell())

builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
parser = Parser(plant)
parser.package_map().Add( "wsg_50_description", os.path.dirname(FindResourceOrThrow(
            "drake/manipulation/models/wsg_50_description/package.xml")))

directives_file = FindResourceOrThrow("drake/sandbox/planar_iiwa_simple_collision_welded_gripper.yaml") \
    if simple_collision else FindResourceOrThrow("drake/sandbox/planar_iiwa_dense_collision_welded_gripper.yaml")
directives = LoadModelDirectives(directives_file)
models = ProcessModelDirectives(directives, plant, parser)


q0 = [-0.2, -1.2, 1.6]
index = 0
for joint_index in plant.GetJointIndices(models[0].model_instance):
    joint = plant.get_mutable_joint(joint_index)
    if isinstance(joint, RevoluteJoint):
        joint.set_default_angle(q0[index])
        index += 1

plant.Finalize()

visualizer = ConnectMeshcatVisualizer(builder, scene_graph, zmq_url=zmq_url, delete_prefix_on_load=False)

diagram = builder.Build()
visualizer.load()
context = diagram.CreateDefaultContext()
plant_context = plant.GetMyContextFromRoot(context)
diagram.Publish(context)

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7008/static/


Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6009...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7008/static/
Connected to meshcat-server.
